In [11]:
#import dependencies 
import random
import copy
from timeit import default_timer as timer
import pprint

## Creating a Texas Holdem Class
- Below a class is made that does several things
1. Hand- creates a hand of two cards and seperates them from the deck so that the simulation can shuffle the remaining 50 cards and create thousands of randoms hands
2. Table- deals the cards out to the correct number of players (for the Winning hand determination) and fills the table with cards (5 2 or 1 card can be swapped out for the simulation)
     - The number of simulated hands should still be relatively high for one card swaps because the number the winning hand class
3. Prob- Identifies if a hand hit any of the possibe draws 
    - Note: I decided that if something is hit and a better hand is hit simultaniously or later the worse hand is still counted as being hit
    - IE: if you have pocket aces you will hit a pair 100% of the time even if some of the time you end up with three of a kind, two pair, or full house. 

In [12]:
class TH:
    class Hand:
        def __init__(self):
            cards = [2,3,4,5,6,7,8,9,10,11,12,13,14]
            suits = ['H','D','S','C']
            self.deck = [(card,suit) for card in cards for suit in suits]
            random.shuffle(self.deck)
        def player_1(self):
            return self.deck[:2]
        def table(self,cards_on_table = 3):
            return self.deck[2:2+cards_on_table], self.deck[2+cards_on_table:]

    class Table:
        def __init__(self, player_1, table, player_count = 5):
            self.player_1 = player_1
            self.table = table[0]
            self.player_count = player_count
            self.hands = player_1[0]
            self.remaining_deck = table[1]
            random.shuffle(self.remaining_deck)

        def deal(self):
            remaining_deck_copy = copy.copy(self.remaining_deck)
            hands = [[] for _ in range(self.player_count-1)]
            for _ in range(2): #hands everyone one card and then a second
                for i in range(self.player_count-1):
                    hands[i].append(remaining_deck_copy.pop())
            table_dealt = [*self.table, *remaining_deck_copy[-(5-len(self.table)):]]
            return hands, table_dealt

    class Prob:
        def __init__(self, player_1, deal, print_hand = False):
            self.player_1 = player_1
            self.other_players = deal[0]
            self.table = deal[1]
            self.total_hand = [*player_1,*self.table]
            self.suits = [x[1] for x in self.total_hand]
            self.value = [x[0] for x in self.total_hand] #list of cards
            if print_hand == True:
                print('Your Cards:')
                print(self.player_1)
                print('Table:')
                print(self.table)

        def flush(self):
            return (self.suits.count('H') >= 5) or (self.suits.count('D') >= 5) or \
        (self.suits.count('S') >= 5) or (self.suits.count('C') >= 5)

        def straight(self):
            #if their is a 14 add a 1 to the begining because Ace is high or low for straight
            cards = list(set(self.value))
            if cards.count(14) == 1:
                cards.append(1)
            cards.sort()
            c = 1
            for i in range(len(cards)-1):
                if cards[i] + 1 == cards[i + 1]:
                    c += 1
                elif c == 5:
                    return True
                else:
                    c = 1
            return (c == 5)
        
        #this needs to be fixed
        def straight_flush(self):
            return False

        def pair(self):
            for i in set(self.value):
                if self.value.count(i) >= 2:
                    return True
            return False

        def three_of_kind(self):
            c = 0
            for i in set(self.value):
                if self.value.count(i) == 3:
                    c += 1
            return (c == 1)

        def two_pair(self):
            c = 0
            for i in set(self.value):
                if self.value.count(i) >= 2:
                    c += 1
            return (c >= 2)

        def four_of_kind(self):
            c = 0
            for i in set(self.value):
                if self.value.count(i) == 4:
                    c += 1
            return (c == 1)

        def full_house(self):
            pair = 0
            three = 0
            for i in set(self.value):
                if self.value.count(i) == 2:
                    pair += 1
                elif self.value.count(i) >= 3:
                    three += 1
            return ((pair >= 1) and (three == 1)) or (three >= 2)

## winning hand

In [21]:
def best_hand(hand,table):
    return hand_ranked(hand)
    

In [ ]:
best_hand()

---

In [13]:
#creating a single hand to itterate over
Game = TH.Hand()
Cards = Game.player_1()
table = Game.table(cards_on_table = 3) # 0 3 or 4
p1 = Cards
print(
    f'Your Cards:\n {p1}',
    f'\nTable:\n {table[0]}'
)

Your Cards:
 [(7, 'D'), (8, 'D')] 
Table:
 [(11, 'S'), (11, 'H'), (4, 'D')]


In [14]:
#Example 
#dependent on what you set the above numbers to you will see a different number of new cards
n_players = 5
Texas_Holdem = TH.Table(Cards,table,n_players)
dealt = Texas_Holdem.deal()

Holdem = TH.Prob(p1,dealt,print_hand = True)

Your Cards:
[(7, 'D'), (8, 'D')]
Table:
[(11, 'S'), (11, 'H'), (4, 'D'), (2, 'H'), (4, 'S')]


In [15]:
def hand_ranked(hand):
    Texas_Holdem = TH.Table(hand,table,n_players)
    dealt = Texas_Holdem.deal()
    Holdem = TH.Prob(p1,dealt)
    return {
        'straight_flush' : Holdem.straight_flush(), #straight flush
        'four_of_kind' : Holdem.four_of_kind(),     #four of a kind
        'full_house' : Holdem.full_house(),         #full house
        'flush' : Holdem.flush(),                   #flush
        'straight' : Holdem.straight(),             #straight
        'three_of_kind' : Holdem.three_of_kind(),   #three of a kind
        'two_pair' : Holdem.two_pair(),             #two pair
        'pair' : Holdem.pair()                      #pair
        }

hand_ranked(Cards)

{'straight_flush': False,
 'four_of_kind': False,
 'full_house': False,
 'flush': False,
 'straight': False,
 'three_of_kind': False,
 'two_pair': True,
 'pair': True}

In [16]:
#look into if you can do this faster by summing dictionary values or using a pandas dataframe? list?
def probabilities(hand, n = 1000):
    straight_flush = 0
    four_of_kind = 0
    full_house = 0
    flush = 0
    straight = 0
    three_of_kind = 0
    two_pair = 0
    pair = 0
    for i in range(n):
        sim_hand = hand_ranked(hand)
        straight_flush += sim_hand['straight_flush'] #straight flush
        four_of_kind += sim_hand['four_of_kind'] #four of a kind
        full_house += sim_hand['full_house'] #full house
        flush += sim_hand['flush'] #flush
        straight += sim_hand['straight'] #straight
        three_of_kind += sim_hand['three_of_kind'] #three of a kind
        two_pair += sim_hand['two_pair'] #two pair
        pair += sim_hand['pair'] #pair
    return {'straight_flush': (straight_flush/n)*100,
            'four_of_kind': (four_of_kind/n)*100,
            'full_house': (full_house/n)*100,
            'flush': (flush/n)*100,
            'straight': (straight/n)*100,
            'three_of_kind': (three_of_kind/n)*100,
            'two_pair': (two_pair/n)*100,
            'pair': (pair/n)*100}

In [17]:
start = timer()

simulated_hands = 100000
prob_of_hitting = probabilities(Cards, n = simulated_hands)

print(f'Your Hand: {p1}\n',
    f'The Table: {table[0]}\n\n',
    f'Probabilities:\n {prob_of_hitting}',
)

end = timer()
print(f'Execution time: {round(end - start,2)}s')

Your Hand: [(7, 'D'), (8, 'D')]
 The Table: [(11, 'S'), (11, 'H'), (4, 'D')]

 Probabilities:
 {'straight_flush': 0.0, 'four_of_kind': 0.117, 'full_house': 2.519, 'flush': 4.03, 'straight': 2.922, 'three_of_kind': 9.273000000000001, 'two_pair': 39.916000000000004, 'pair': 100.0}
Execution time: 5.35s


In [18]:
prob_of_hitting = probabilities(Cards, n = simulated_hands)


In [19]:
prob_of_hitting['straight_flush']

0.0

In [20]:
prob_of_hitting

{'straight_flush': 0.0,
 'four_of_kind': 0.089,
 'full_house': 2.4619999999999997,
 'flush': 4.167,
 'straight': 2.935,
 'three_of_kind': 9.223,
 'two_pair': 39.985,
 'pair': 100.0}